<span style="display: inline-block; margin-right: 10px; vertical-align: middle; border: 1px solid #F8F8F8; border-radius: 8px; padding: 5px; background-color: #F8F8F8;">
    <a href="https://www.designsafe-ci.org/workspace/jupyter-lab-hpc-cuda-ds?appVersion=1.1.1" target="_parent" style="text-decoration: none;">
        <span style="font-family: Helvetica, sans-serif; font-size: 13px; color: #565656; margin-right: 5px; vertical-align: middle; font-weight: 600;">Open in</span>
        <img src="https://www.designsafe-ci.org/media/filer_public/2d/d3/2dd37fbf-289e-49cf-9c1a-879c864c4e17/nsf_nheri-ds.png" alt="Open in DesignSafe" style="width: 100px; height: auto; vertical-align: middle;">
    </a>
</span>

<span style="display: inline-block; margin-right: 10px; vertical-align: middle;">
    <a href="https://colab.research.google.com/github/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/chimney_detector.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width: 130px; height: auto; vertical-align: middle;"/>
    </a>
</span>

<span style="display: inline-block; vertical-align: middle; margin-top: 6px;">
    <a target="_blank" href="https://lightning.ai/new?repo_url=https%3A//github.com/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/chimney_detector.ipynb">
        <img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/app-2/studio-badge.svg" alt="Open in Studio" style="width: 120px; height: auto; vertical-align: middle;"/>
    </a>
</span>

## **Install BRAILS++**
BRAILS++ is not yet available on PyPi. The following line installs the latest version from the GitHub repository using `pip`.

In [1]:
!pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus

  Cloning https://github.com/NHERI-SimCenter/BrailsPlusPlus to /tmp/pip-req-build-p2vgpyab
  Running command git clone --filter=blob:none --quiet https://github.com/NHERI-SimCenter/BrailsPlusPlus /tmp/pip-req-build-p2vgpyab
  Resolved https://github.com/NHERI-SimCenter/BrailsPlusPlus to commit dd8347005551cac51fef125d9d99429e8c2abebd
  Preparing metadata (setup.py) ... done
  Using cached wcwidth-0.2.13-py2.py3-none-any.whl.metadata (14 kB)
Using cached wcwidth-0.2.13-py2.py3-none-any.whl (34 kB)
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.5
    Uninstalling wcwidth-0.2.5:
      Successfully uninstalled wcwidth-0.2.5
Note: you may need to restart the kernel to use updated packages.


## **Import ImageSet and Importer Methods of BRAILS++**
BRAILS++ offers various Artificial Intelligence-powered modules for predicting building and infrastructure attributes from images, such as the `ChimneyDetector` demonstrated in this example.

The `Importer` class provides the main methods to load these modules. To gather the images needed for attribute prediction, users can use the ImageSet class or BRAILS++ scrapers, which facilitate automated asset geometry and image retrieval. This example illustrates both methods for sourcing imagery for use with the `ChimneyDetector` module.

In [2]:
from brails import Importer
from brails.types.image_set import ImageSet

## Define Google API Key
You need a Google API Key with Street View Static API enabled to download the Google Street View imagery required for this example. Please follow this [link](https://support.google.com/googleapi/answer/6158862?hl=en) for instructions on setting up a Google API key.

In [6]:
API_KEY = 'YOUR-API-KEY-HERE'

## **Load Street-Level Imagery: (Option 1) Download Imagery by Specifying the Region Name**
The `ChimneyDetector` class is created to identify the existence of chimneys in buildings from street-level imagery. In this option, BRAILS++ retrieves the required imagery by following the steps below.
1. <strong>Call the `RegionBoundary` class with the necessary details to specify the region of interest.</strong> In this example, street-level imagery of buildings in Atlantic County, NJ, are downloaded. Therefore, the required information includes the `type` of information provided and the `data` itself, which are the `locationName` and `Atlantic County, NJ`.

2. <strong>Download the footprint inventory for the `RegionBoundary` saved in `region_boundary_object` by downloading the OpenStreetMap footprint data through the `OSM_FootprintScraper`.</strong> The only information required to initialize `OSM_FootprintScraper` is the desired output unit for `length`, which, in this case, is `ft`. The output of `OSM_FootprintScraper` is an `AssetInventory` saved in `atlantic_fp`.

3. <strong>Get a randomly selected `100` building subset of the footprint inventory obtained using `OSM_FootprintScraper`. </strong> This subset is obtained using the `get_random_sample` method of an `AssetInventory` object and is stored in `atlantic_fp_subset`. For this example, the random seed is set to `40` for reproducibility of the results.

4. <strong>Get the street-level imagery for the selected subset using `get_images` method of `GoogleStreet` module.</strong> `get_images` requires two inputs 1) AssetInventory for which the images will be retrieved (in this example, `atlantic_fp_subset`) and 2) the path to the folder where the retrieved images will be saved, which in this case is `tmp/street/`.


In [8]:
# Select a region and create its RegionBoundary:
importer = Importer()

region_data = {"type": "locationName",
               "data": "Atlantic County, NJ"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

osm_fp_class = importer.get_class("OSM_FootprintScraper")
osm_fp_data = {"length": "ft"}
footprint_scraper = osm_fp_class(osm_fp_data)
atlantic_fp = footprint_scraper.get_footprints(region_boundary_object)

# Subsample from the extracted assets to keep the image downloading step quick.
# Here, we are randomly sampling 100 buildings using a random seed value of 40:
atlantic_fp_subset = atlantic_fp.get_random_sample(100, 40)

# Get street-level imagery for the selected subset using GoogleStreetview:
google_street_class = importer.get_class("GoogleStreetview")
google_street = google_street_class({'apiKey': API_KEY})
images_street = google_street.get_images(
    atlantic_fp_subset, 'tmp/street/')


Searching for Atlantic County, NJ...
Found Atlantic County, New Jersey, United States

Found a total of 18900 building footprints in Atlantic County


INFO:brails.scrapers.google_streetview.google_streetview:
Images will be saved to: /home/bacetiner/Documents/BrailsPlusPlus/examples/image_processor/tmp/street

Obtaining street-level imagery:   1%|█▏                                                                                                                       | 1/100 [00:00<00:19,  5.05it/s]INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.4948, -74.4877
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.3586, -74.6355
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.4565, -74.6337
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39.4438, -74.6872
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 39

## **Predict Existence of Chimneys in A Building Using the ChimneyDetector Module**
`ChimneyDetector` includes a pre-trained EfficientDet_D4 model that has been trained on a custom dataset of 40,000 images. This model is capable of identifying the existence of chimneys.

In [9]:
importer = Importer()
chimney_detector_class = importer.get_class('ChimneyDetector')
chimney_detector = chimney_detector_class()
predictions = chimney_detector.predict(images_street)

NotFoundError: Class ChimneyDetector is not found.
These are the available classes: {'BrailsError': 'brails.exceptions', 'NotFoundError': 'brails.exceptions', 'FoundationHeightClassifier': 'brails.processors.FoundationClassifier.FoundationClassifier', 'NFloorDetector': 'brails.processors.nfloors_detector.nfloor_detector', 'RoofShapeVLM': 'brails.processors.roof_shape_vlm.roof_shape_vlm', 'BuildingMaterialLLM': 'brails.processors.building_material_llm.building_material_llm', 'NFloorVLM': 'brails.processors.nFloor_vlm.nFloor_vlm', 'BoundingBox': 'brails.processors.vlm_segmenter.grounded_sam_utils', 'DetectionResult': 'brails.processors.vlm_segmenter.grounded_sam_utils', 'VLMSegmenter': 'brails.processors.vlm_segmenter.vlm_segmenter', 'CLIPClassifier': 'brails.processors.vlm_image_classifier.CLIPClassifier', 'GarageDetector': 'brails.processors.garage_detector.garage_detector', 'ImageClassifier': 'brails.processors.image_classifier.image_classifier', 'DatasetBinary': 'brails.processors.im

## **Retrain Default ChimneyDetector Module**

In [ ]:
chimney_detector.retrain(images_street)